# Comparison for LogSumExp

The LogSumExp-problem with linear consraints:
	$$\log(1 + \sum\limits_{k=1}^n \exp(\alpha x_k)) + \frac{\beta}{2} \|\textbf{x}\|_2^2 \rightarrow \min_{\textbf{x}\in \mathbb{R}^N}$$
	$$\text{s.t.} (\textbf{b}_j, \textbf{x}) - c_j <= 0, j = 1\dots M$$
	
This problem can be converted into saddle point problem:
	$$\min_{\textbf{x}} \max_{\textbf{y}\in\mathbb{R}^M_+} S(\textbf{x},\textbf{y}),$$
where $S(\textbf{x}, \textbf{y}) = r(\textbf{x})+F(\textbf{x}, \textbf{y}) - h(\textbf{y})$
	$$r(\textbf{x}) = \log(1 + \sum\limits_{k=1}^n \exp(\alpha x_k)) + \frac{\beta}{2} \|\textbf{x}\|_2^2$$
	$$F(\textbf{x}, \textbf{y}) = \sum\limits_{j=1}^M x_j(\textbf{b}_j, \textbf{y}) = (\textbf{x}, B\textbf{y})$$
	$$h(\textbf{y}) = (\textbf{c}, \textbf{y})$$

So N is dimensional of external variable, M - internal. But we will consider the external problem as help problem.

In [1]:
import numpy as np
from TestFunctions import LogSumExp_difalpha as LogSumExp
from TestFunctions.TestFunctions import TestFunction
from Solvers import GradientMethods
from Solvers.HalvCube import Dichotomy_exact
from Solvers.GradientMethods import FGM_internal
from Solvers.Ellipsoids import ellipsoid
import matplotlib.pyplot as plt
%matplotlib notebook
import comparison
import imp

from comparison import method_comparison, create_methods_dict

In [2]:
def get_LSE(N = 100, M = 2, alpha = 0.001, beta = 0.5e-3, k = 1, eps = 1e-3):
    """
    The function is to get class of LSE. All parameters mathes the
    parameters above.
    
    N is dimensional of primal problem
    M is number of constraints. It is also the dual dimension
    alpha is parameter of LogSumExp problem
    beta is coefficient of l2-regularization for primal problem
    
    k is parameter for to generate matrix B
    """
    np.random.seed(N*M//13)
    alpha_ = np.random.uniform(size = (N,), low = -alpha, high = alpha)
    alpha = np.max(np.abs(alpha_))
    B = []
    for i in range(M):
        b = np.random.uniform(low = -k, high = 0, size = (N,)) * np.sign(alpha_)
        p = 1
        z = np.array(np.random.uniform(low = 0, high = 1, size = (N,))<=p, dtype = np.float32)
        B.append(b * z)
    B = np.array(B)
    #B = np.random.uniform(low = -k, high = 0, size = (M, N))
    c = np.ones((M,))
    x_ = np.zeros((N,))
    f_ = LogSumExp.r(alpha, beta).get_value(x_)
    gamma = c.max()
    y_max = f_/gamma
    Q = [[0, y_max] for i in range(M)]
    Q = np.array(Q)
    size_domain = np.linalg.norm(Q[:,0]-Q[:,1])
    solver = FGM_internal
    Q = Q.copy()
    beta_ = 2 * eps/ (size_domain**2)
    LSE_y_external = TestFunction(r = LogSumExp.h(c, beta = beta_, size_domain = size_domain, y0 = (Q[:,0] + Q[:, 1])/2),
                    F = LogSumExp.F_tilde(B, size_domain),
                    h = LogSumExp.r(alpha_, beta, size_domain),
                    solver = solver, 
                    get_start_point = lambda x: (1/beta * -x.dot(B), alpha * np.sqrt(N)/beta))
    solver = ellipsoid
    Q = Q.copy()
    LSE_ellipsoid = TestFunction(r = LogSumExp.r(alpha_, beta, size_domain),
                    F = LogSumExp.F(B, size_domain),
                    h = LogSumExp.h(c, beta = beta_, size_domain = size_domain, y0 = (Q[:,0] + Q[:, 1])/2),
                    solver = solver, 
                    get_start_point = lambda x: ((Q[:,0] + Q[:, 1])/2, size_domain/2))
    solver = lambda func, grad, L, mu, start_point, cond: Dichotomy_exact().Halving(func, 
                                                                                    grad, 
                                                                                    L, mu, 
                                                                                    eps, 
                                                                                    start_point,
                                                                                    cond)
    Q = Q.copy()
    LSE_dichotomy = TestFunction(r = LogSumExp.r(alpha_, beta, size_domain),
                F = LogSumExp.F(B, size_domain),
                h = LogSumExp.h(c, beta = beta_, size_domain = size_domain, y0 = (Q[:,0] + Q[:, 1])/2),
                solver = solver, 
                get_start_point = lambda x: ((Q[:,0] + Q[:, 1])/2, size_domain/2))
    solver = FGM_internal
    Q = Q.copy()
    LSE_fgm = TestFunction(r = LogSumExp.r(alpha_, beta, size_domain),
                F = LogSumExp.F(B, size_domain),
                h = LogSumExp.h(c, beta = beta_, size_domain = size_domain, y0 = (Q[:,0] + Q[:, 1])/2),
                solver = solver, 
                get_start_point = lambda x: ((Q[:,0] + Q[:, 1])/2, size_domain/2))
    R = alpha * np.sqrt(N)/beta
    return (LSE_y_external, Q), (LSE_ellipsoid, R), (LSE_dichotomy, R), (LSE_fgm, R), (alpha_, B, c, beta)

In [3]:
def start_comparison(N = 100, M = 2, alpha = 0.001, beta = 1e-6, k = 100, eps = 1e-3, time_max = 0.2):
    LSE_res = get_LSE(N = N, M = M, alpha = alpha, beta = beta, k = k, eps = eps)
    (LSE_y_external, Q), (LSE_ellipsoid, R), (LSE_dichotomy, R), (LSE_fgm, R), stop_cond_args = LSE_res
    keys = {"FGM":"y, x - FGM+FGM",
        "Ellipsoids": "y, x - Ellipsoids+FGM",
        "Dichotomy": "y, x - Dichotomy+FGM"}
    #keys = {"FGM":"y, x - FGM+FGM",
    #    "Ellipsoids": "y, x - Ellipsoids+FGM"}
    history = {}
    f_dict = {}
    methods = {}
    methods = create_methods_dict(LSE_y_external, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = time_max, stop_cond_args = stop_cond_args, keys = keys)
    for key in methods:
        cond = lambda y, R: LSE_y_external.L_xx * R <= 1e-3
        f = lambda y: LSE_y_external.get_value(y, LSE_y_external.get_solution(y, cond)[0])
        f_dict[key] = f
    Q = np.vstack([np.zeros((N, )) - R * np.ones((N,)), np.zeros((N, )) + R * np.ones((N,))]).T
    methods["x, y - FGM+FGM"] = create_methods_dict(LSE_fgm, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = time_max, inverse = True,  stop_cond_args = stop_cond_args, keys = {"FGM":"x, y - FGM+FGM"})["x, y - FGM+FGM"]
    cond = lambda y, R: LSE_fgm.L_xx * R <= eps
    f = lambda y: LSE_fgm.get_value(y, LSE_fgm.get_solution(y, cond)[0])
    f_dict["x, y - FGM+FGM"] = f
    """

    methods["x, y - FGM+Ellipsoids"] = create_methods_dict(LSE_ellipsoid, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = time_max, inverse = True,  stop_cond_args = stop_cond_args, keys = {"FGM":"x, y - FGM+Ellipsoids"})["x, y - FGM+Ellipsoids"]
    cond = lambda y, R: LSE_ellipsoid.L_yy * R <= eps
    f = lambda y: LSE_ellipsoid.get_value(y, LSE_ellipsoid.get_solution(y, cond)[0])
    f_dict["x, y - FGM+Ellipsoids"] = f
    Q = np.vstack([np.zeros((N, )) - R * np.ones((N,)), np.zeros((N, )) + R * np.ones((N,))]).T
    methods["x, y - FGM+Dichotomy"] = create_methods_dict(LSE_dichotomy, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = time_max, inverse = True, stop_cond_args = stop_cond_args, keys = {"FGM":"x, y - FGM+Dichotomy"})["x, y - FGM+Dichotomy"]
    cond = lambda y, R: LSE_dichotomy.L_xx * R <= eps
    """
    f = lambda y: LSE_dichotomy.get_value(y, LSE_dichotomy.get_solution(y, cond)[0])
    f_dict["x, y - FGM+Dichotomy"] = f
    #comparison.method_comparison(methods)
    for key in methods:
        methods[key]()
    return history, stop_cond_args

In [4]:
def parse_results(history, alpha_, B, c, beta, keys = ["Ellipsoids", "Dichotomy", "FGM"]):
    s_key = ""
    s_feas = ""
    s_time = ""
    s_value = ""
    for key in keys:
        s_key += key + "\t$"
        if "y, x" in key:
            lambda_, x = history[key][-1][0]
        else:
            x, lambda_ = history[key][-1][0]
        s_feas += str((B @ x <= c).any()) + "\t\t$"
        s_time += "%.3f"%(history[key][-1][1] - history[key][0][1]) + "\t\t$"
        x_ = alpha_ * x
        M = np.max(x_)
        x_ = x_ - M
        s_value+= "%.3f"%(M + np.log(np.exp(-M) + np.exp(x_).sum()) + beta * np.linalg.norm(x)**2) + "\t\t$"
    print(s_key)
    print(s_feas)
    print(s_time)
    print(s_value)

## M= 2

In [5]:
M = 2

### N = 100

In [6]:
N = 100

In [7]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 10)
alpha_, B, c, beta = stop_cond_args

/home/elias/Desktop/PROJECTS/SPP/SPP/Solvers/GradientMethods.py:40: RuntimeWarning: divide by zero encountered in double_scalars
  est = 2 * min(4 * L * R/N**2, L * R * np.exp(-N/2 * np.sqrt(mu/L)))


In [8]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
history["y, x - Dichotomy+FGM"][-1][1] - history["y, x - Dichotomy+FGM"][-2][1]

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$True		$True		$False		$
0.024		$0.112		$0.049		$10.000		$
4.615		$4.792		$5.518		$4.615		$


0.0013880729675292969

In [9]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [10]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
history["y, x - Dichotomy+FGM"][-1][1] - history["y, x - Dichotomy+FGM"][-2][1]

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$True		$True		$False		$
24.400		$0.290		$40.003		$40.001		$
4.615		$4.615		$4.615		$4.615		$


0.003991127014160156

In [11]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M,
                                           eps = eps,
                                           time_max = 5)
alpha_, B, c, beta = stop_cond_args

In [12]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
history["y, x - Dichotomy+FGM"][-1][1] - history["y, x - Dichotomy+FGM"][-2][1]

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$False		$True		$True		$
5.003		$0.614		$5.005		$5.001		$
4.615		$4.615		$4.615		$4.615		$


0.00949716567993164

### N = 1000

In [13]:
M=2
N = 1000

In [14]:
eps = 1e-7
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 1)
alpha_, B, c, beta = stop_cond_args

In [15]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
len(history["y, x - Dichotomy+FGM"]), len(history["y, x - FGM+FGM"])

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$True		$True		$True		$
1.001		$0.808		$0.469		$1.001		$
6.909		$6.909		$6.909		$6.909		$


(76, 81)

In [16]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 1)
alpha_, B, c, beta = stop_cond_args

In [17]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
len(history["y, x - Dichotomy+FGM"]), len(history["y, x - FGM+FGM"])

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$True		$True		$True		$
1.005		$0.536		$0.305		$1.000		$
6.909		$6.909		$6.909		$6.909		$


(65, 94)

In [19]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 20)
alpha_, B, c, beta = stop_cond_args

In [20]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
history["y, x - Dichotomy+FGM"][-1][1] - history["y, x - Dichotomy+FGM"][-2][1]

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$True		$True		$True		$
20.001		$1.896		$20.001		$20.001		$
6.909		$6.909		$6.909		$6.909		$


0.01411581039428711

## M= 3

In [21]:
M = 3

### N = 100

In [22]:
N = 100

In [23]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

/home/elias/Desktop/PROJECTS/SPP/SPP/TestFunctions/LogSumExp_difalpha.py:46: RuntimeWarning: overflow encountered in exp
  return alpha * np.exp(x_)/ (np.exp(-x_max) + np.exp(x_).sum()) + beta*x


In [24]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
history["y, x - Dichotomy+FGM"][-1][1] - history["y, x - Dichotomy+FGM"][-2][1]

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$True		$True		$False		$
0.014		$0.304		$0.095		$40.000		$
4.615		$4.881		$6.356		$4.615		$


0.0010523796081542969

In [25]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [26]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
history["y, x - Dichotomy+FGM"][-1][1] - history["y, x - Dichotomy+FGM"][-2][1]

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$False		$True		$False		$
30.083		$0.743		$40.002		$40.001		$
4.615		$4.615		$4.615		$4.615		$


0.004060268402099609

In [27]:
y, x = history["y, x - Dichotomy+FGM"][-1][0]
y

array([3.36971728e-09, 5.15779911e-08, 3.43853274e-08])

In [28]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [29]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
history["y, x - Dichotomy+FGM"][-1][1] - history["y, x - Dichotomy+FGM"][-2][1]

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$True		$True		$True		$
40.006		$1.843		$40.007		$40.000		$
4.615		$4.615		$4.615		$4.615		$


0.007762908935546875

### N = 1000

In [30]:
N = 1000

In [31]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [32]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
history["y, x - Dichotomy+FGM"][-1][1] - history["y, x - Dichotomy+FGM"][-2][1]

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$True		$True		$False		$
0.031		$0.622		$0.194		$40.001		$
6.909		$6.925		$6.986		$6.909		$


0.0023660659790039062

In [33]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [34]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
history["y, x - Dichotomy+FGM"][-1][1] - history["y, x - Dichotomy+FGM"][-2][1]

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$True		$True		$False		$
5.207		$1.445		$0.492		$40.001		$
6.909		$6.909		$6.909		$6.909		$


0.008520841598510742

In [35]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [36]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
history["y, x - Dichotomy+FGM"][-1][1] - history["y, x - Dichotomy+FGM"][-2][1]

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$True		$True		$True		$
40.013		$3.503		$40.011		$40.001		$
6.909		$6.909		$6.909		$6.909		$


0.014456510543823242

## M= 4

In [37]:
M = 4

### N = 100

In [38]:
N = 100

In [39]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [40]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
history["y, x - Dichotomy+FGM"][-1][1] - history["y, x - Dichotomy+FGM"][-2][1]

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$True		$True		$False		$
0.013		$0.461		$0.147		$40.001		$
4.615		$4.850		$5.268		$4.615		$


0.0014693737030029297

In [41]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [42]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
history["y, x - Dichotomy+FGM"][-1][1] - history["y, x - Dichotomy+FGM"][-2][1]

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$True		$True		$False		$
31.091		$1.101		$40.001		$40.001		$
4.615		$4.615		$4.615		$4.615		$


0.004576683044433594

In [43]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [44]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
history["y, x - Dichotomy+FGM"][-1][1] - history["y, x - Dichotomy+FGM"][-2][1]

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$True		$True		$True		$
40.002		$2.722		$40.004		$40.001		$
4.615		$4.615		$4.615		$4.615		$


0.0080413818359375

### N = 1000

In [45]:
N = 1000

In [46]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [47]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
history["y, x - Dichotomy+FGM"][-1][1] - history["y, x - Dichotomy+FGM"][-2][1]

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$True		$True		$False		$
0.024		$1.143		$0.280		$40.001		$
6.909		$6.914		$6.939		$6.909		$


0.0030040740966796875

In [48]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [49]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
history["y, x - Dichotomy+FGM"][-1][1] - history["y, x - Dichotomy+FGM"][-2][1]

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$True		$True		$True		$
4.894		$2.588		$0.718		$40.001		$
6.909		$6.909		$6.909		$6.909		$


0.008772850036621094

In [50]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [51]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())
history["y, x - Dichotomy+FGM"][-1][1] - history["y, x - Dichotomy+FGM"][-2][1]

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$
False		$True		$True		$True		$
40.006		$5.368		$40.004		$40.000		$
6.909		$6.909		$6.909		$6.909		$


0.01441049575805664

In [52]:
len(history["y, x - Dichotomy+FGM"])

2635

In [53]:
len(history["y, x - FGM+FGM"])

2853